## 성능
- 54% 정확도 
- https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

### 모델 discription을 일일이 정의해서 학습한 코드


- <-> 사전학습 모델사용은 _VGG 코드파일보기


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 2
batch_size = 2

In [4]:
transform = transforms.Compose([transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)        
       #num_workers: data 로딩을 위해 몇 개의 서브 프로세스를 사용할 것인지를 결정. default=0

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [12]:
# 편의를 위해 relu를 정의하지 않고, pool이 2번 사용되므로 한번만 정의
# 모델 설계, 모델학습 전부 동일

import torch.nn.functional as F   #relu 사용을 위해서

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
                                                      # CIFAR-10 해상도: 32*32*3
        self.conv1 = torch.nn.Conv2d(3, 6, 5)         # 28*28
        self.pool = torch.nn.MaxPool2d(2, 2)          # 14*14
        self.conv2 = torch.nn.Conv2d(6, 16, 5)        # 10*10

        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1, 16 * 5 * 5)     # view(x.size(0),-1) 
                                       # -1(전부)를 16*5*5=400 의 1D데이터로 변형 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# model =Net
net = Net().to(device)

In [13]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

In [14]:
# LeNet5. LeNet5까지는 사용하지 않아도 된다.


for epoch in range(training_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
       
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# [2, 24000] loss: 1.336

[1,  2000] loss: 2.184
[1,  4000] loss: 1.885
[1,  6000] loss: 1.721
[1,  8000] loss: 1.652
[1, 10000] loss: 1.640
[1, 12000] loss: 1.542
[1, 14000] loss: 1.535
[1, 16000] loss: 1.507
[1, 18000] loss: 1.456
[1, 20000] loss: 1.474
[1, 22000] loss: 1.426
[1, 24000] loss: 1.422
[2,  2000] loss: 1.364
[2,  4000] loss: 1.348
[2,  6000] loss: 1.329
[2,  8000] loss: 1.313
[2, 10000] loss: 1.353
[2, 12000] loss: 1.335
[2, 14000] loss: 1.284
[2, 16000] loss: 1.317
[2, 18000] loss: 1.330
[2, 20000] loss: 1.281
[2, 22000] loss: 1.334
[2, 24000] loss: 1.279
Finished Training


In [15]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 53 %
